<a href="https://colab.research.google.com/github/DigdinLogistico/API-1-FATEC/blob/main/SRINT_2_CSV_TRATADO_MUN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

caminho = '/content/drive/My Drive/DATA/COMEXSTAT/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Função auxiliar para carregar múltiplos CSVs rapidamente
def carregar_arquivos(nomes, prefixo=""):
    return {
        nome.lower(): pd.read_csv(f"{caminho}{prefixo}{nome.upper()}.csv", sep=";", encoding="latin1")
        for nome in nomes
    }


In [ ]:
# Arquivos para o município
arquivos_municipio = ["exp_2023_mun", "exp_2024_mun", "exp_2025_mun", "imp_2023_mun", "imp_2024_mun", "imp_2025_mun"]
arquivos_auxiliares = ["ncm_sh", "pais", "uf_mun"]

# Carregar os dados do municipio
dados_municipio = carregar_arquivos(arquivos_municipio)

# Carregar dados auxiliares
dados_auxiliares = carregar_arquivos(arquivos_auxiliares)

# Verificar as primeiras linhas dos dados carregados para conferirmos se está correto
print(dados_municipio["exp_2023_mun"].head())

   CO_ANO  CO_MES   SH4  CO_PAIS SG_UF_MUN   CO_MUN  KG_LIQUIDO  VL_FOB
0    2023      10  2209      127        BA  2919553          36      53
1    2023      11  4009      493        SP  3456701         473   14008
2    2023      11  8536      365        SP  3438709           2     181
3    2023       8  8512       63        RJ  3304201       15627  239105
4    2023       6  1902      580        PR  4118204         287     368


In [ ]:
# Função para processar os dados de estado
def processar_municipio(dados_municipio, dados_auxiliares):
    df_municipio = pd.concat([dados_municipio[f"exp_{ano}_mun"] for ano in range(2023, 2026)], ignore_index=True)
    df_municipio = pd.concat([df_municipio, pd.concat([dados_municipio[f"imp_{ano}_mun"] for ano in range(2023, 2026)], ignore_index=True)], ignore_index=True)

    # Renomear colunas conforme o dicionário
    renomear_colunas_estado = {
        "CO_ANO":"Ano",
        "CO_MES":"Mês",
        "SH4":"Produto",
        "CO_PAIS":"País",
        "SG_UF_MUN":"Estado",
        "CO_MUN":"Municipio",
        "KG_LIQUIDO":"Peso",
        "VL_FOB":"Valor US$",
    }
    df_municipio = df_municipio.rename(columns=renomear_colunas_estado)

    # Merge com as colunas auxiliares definidas
    df_municipio = df_municipio.merge(dados_auxiliares["pais"][["CO_PAIS", "NO_PAIS"]].rename(columns={"CO_PAIS": "País", "NO_PAIS": "Nome do País"}), on="País", how="left")

    df_municipio = df_municipio.merge(dados_auxiliares["uf_mun"][["CO_MUN_GEO", "NO_MUN_MIN"]].rename(columns={"CO_MUN_GEO": "Municipio", "NO_MUN_MIN": "Nome da Cidade"}), on="Municipio", how="left")


    return df_municipio

# Processar estado
df_municipio_processado = processar_municipio(dados_municipio, dados_auxiliares)

# Salvar arquivo processado
df_municipio_processado.to_csv('/content/drive/My Drive/DATA/COMEXSTAT/municipio_processado_1.0.csv', index=False)

In [ ]:
# Visualização para debug
df_municipio = df_municipio.merge(dados_auxiliares["ncm_sh"][["CO_SH4", "NO_SH4_POR"]].rename(columns={"CO_SH4": "Produto", "NO_SH4_POR": "Nome do Produto"}), on="Produto", how="left") # Corrected merge columns and rename

In [ ]:
print(df_municipio_processado.head())

    Ano  Mês  Produto  País Estado  Municipio   Peso  Valor US$ Nome do País  \
0  2023   10     2209   127     BA    2919553     36         53   Cabo Verde   
1  2023   11     4009   493     SP    3456701    473      14008       México   
2  2023   11     8536   365     SP    3438709      2        181    Indonésia   
3  2023    8     8512    63     RJ    3304201  15627     239105    Argentina   
4  2023    6     1902   580     PR    4118204    287        368       Panamá   

           Nome da Cidade  
0  Luís Eduardo Magalhães  
1                 Vinhedo  
2              Piracicaba  
3                 Resende  
4               Paranaguá  


In [ ]:
# Excluir colunas especificadas
colunas_para_excluir = [ "País", "Municipio"]
df_municipio_processado = df_municipio_processado.drop(columns=colunas_para_excluir, errors='ignore')

# Mapear números para que correspondam aos seus respectivos números
mapa_mensal = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}
df_municipio_processado['Mês'] = df_municipio_processado['Mês'].map(mapa_mensal)

# Pré visualização do dataframe tratado
display(df_municipio_processado.head())

,Ano,Mês,Produto,Estado,Peso,Valor US$,Nome do País,Nome da Cidade
0,2023,Outubro,2209,BA,36,53,Cabo Verde,Luís Eduardo Magalhães
1,2023,Novembro,4009,SP,473,14008,México,Vinhedo
2,2023,Novembro,8536,SP,2,181,Indonésia,Piracicaba
3,2023,Agosto,8512,RJ,15627,239105,Argentina,Resende
4,2023,Junho,1902,PR,287,368,Panamá,Paranaguá


In [ ]:
# Salvar o DataFrame em csv
df_municipio_processado.to_csv('/content/drive/My Drive/DATA/COMEXSTAT/municipio_processado_1.2.csv', index=False)